In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [9]:
root_dir="C:\College stuff'\Sem4\Mini Project\Main"
number_of_images={}

for dir in os.listdir(root_dir):
    number_of_images[dir] = len(os.listdir(os.path.join(root_dir, dir)))

 

In [10]:
number_of_images.items()

dict_items([('Healthy', 3494), ('Tumor', 6517)])

We Will split the data such that 
70% is train data 
15% for validation 
15% for validaton

In [11]:
def datafolder(p,split):
    
    if not os.path.exists('./'+p):
     os.mkdir("./"+p)

    for dir in os.listdir(root_dir):
        os.makedirs('./'+p+"/"+dir)
        for img in np.random.choice(a=os.listdir(os.path.join(root_dir,dir)), size=(math.floor(split*number_of_images[dir])-2),replace=False):

         O=os.path.join(root_dir,dir,img)
         D=os.path.join('./'+p, dir)
         shutil.copy(O,D)
         os.remove(O)
    else:
       print("folder exists")
        

In [12]:
datafolder("Train",0.4)

folder exists


In [13]:
datafolder("Test",0.2)


folder exists


In [14]:
datafolder("val",0.2)

folder exists


In [141]:
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization,GlobalAveragePooling2D
from keras.models import Sequential
from tensorflow.keras.utils import load_img
import keras
from keras.preprocessing.image import ImageDataGenerator


In [142]:
model=Sequential()

model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))

model.add(Conv2D(filters=366, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1,activation='relu'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 conv2d_17 (Conv2D)          (None, 220, 220, 366)     53070     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 110, 110, 366)    0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 108, 108, 64)      210880    
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 52, 52, 128)      

In [143]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [144]:
def preprocessingimages(path):
    image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255, horizontal_flip=True)
    image=image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')
    return image

In [145]:
path="C:\College stuff'\Sem4\Mini Project\Train"
train_data=preprocessingimages(path)

Found 1796 images belonging to 2 classes.


In [146]:
def preprocessingimages2(path):
    image_data=ImageDataGenerator(rescale=1/255)
    image=image_data.flow_from_directory(directory=path, target_size=(224,224), batch_size=32, class_mode='binary')
    return image

In [147]:
path="C:\College stuff'\Sem4\Mini Project\Test"
test_data=preprocessingimages2(path)

Found 596 images belonging to 2 classes.


In [148]:
path="C:\College stuff'\Sem4\Mini Project\Val"
val_data=preprocessingimages2(path)

Found 596 images belonging to 2 classes.


In [149]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Early Stopping
ec=EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=6, verbose=1, mode='auto')

#model checpoint
mc=ModelCheckpoint(monitor="val_accuracy", filepath="./bestmodel.h5", verbose=1, save_best_only=True, mode='auto')

cd=[ec,mc]